In [1]:
import mdtraj

from grand import lig_samplers as ls
from grand import lig_utils as lu

import openmm
import openmm.app
import openmm.unit
from pdbfixer import PDBFixer

import openff.toolkit

from openmmtools.integrators import VelocityVerletIntegrator
from openmmtools.integrators import BAOABIntegrator

import numpy as np
import mdtraj as md
from scipy.spatial import Voronoi, Delaunay
import os
from collections import defaultdict
import tqdm
import pickle

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
datadir = '../../examples/lysozyme/'
prot = openmm.app.PDBFile(f'{datadir}/apo.pdb')

In [3]:
mol = openff.toolkit.Molecule.from_smiles("CC1=CC=CC=C1")
mol.generate_conformers(n_conformers=1000)
topology = mol.to_topology().to_openmm()
positions = mol.conformers[0].to_openmm()

In [4]:
if os.path.exists('gcmc-ghosts-1000.pdb'):
    pdb = openmm.app.PDBFile('gcmc-ghosts-1000.pdb')
    sys_top = pdb.topology
    sys_pos = pdb.positions
else:
    sys_top, sys_pos, ghosts = lu.add_ghosts(prot.topology, prot.positions, topology, positions, n=1000,output=f'{datadir}/gcmc-ghosts-1000.pdb')

In [5]:
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml',f'{datadir}/toluene_openff.xml')

In [6]:
pdb = openmm.app.PDBFile('gcmc-ghosts-1000.pdb')

In [7]:
pdb_system = ff.createSystem(pdb.topology)
integrator = openmm.LangevinIntegrator(298.0 * openmm.unit.kelvin, 1.0 / openmm.unit.picosecond, 2.0 * openmm.unit.femtosecond)
reporter = openmm.app.DCDReporter(f"{datadir}/sacp_lysozyme.dcd", reportInterval=1000)

In [8]:
frag_res_ids = []
for resid, residue in enumerate(pdb.topology.residues()):
    if residue.name == "UNK":
        frag_res_ids.append(resid)

In [9]:
sampler = ls.GCNCMCSampler(ff,pdb.topology,298*openmm.unit.kelvin,ligands=frag_res_ids,overwrite=True,B=0,positions=pdb.positions,
                           integrator=integrator,reporters=[reporter], ghosts=frag_res_ids, insert_prob=1.0, n_pert_steps=1, n_prop_steps_per_pert=100)

HarmonicBondForce
1
37332.0680475235 kJ/mol
PeriodicTorsionForce
2
14778.185791015625 kJ/mol
CMMotionRemover
3
0.0 kJ/mol
HarmonicAngleForce
4
101701.98937988281 kJ/mol
CustomNonbondedForce
5
nan kJ/mol
CustomBondForce
6
28740.502807617188 kJ/mol
Initial Energy: nan kJ/mol


In [10]:
for n in tqdm.tqdm(range(10)):
    sampler.move()
print(sampler.n_accepted)

100%|██████████| 10/10 [00:03<00:00,  3.06it/s]

0
